<a href="https://colab.research.google.com/github/LeeDongHun38/recommendation_LLM/blob/main/warm_start_b3d2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Warm Start 추천용 NeuMF 모델 제작**

🎯 **단계 목표**

* NeuMF의 핵심인 GMF(선형) 경로와 MLP(비선형) 경로를 모두 갖춘 하이브리드 모델을 정의한다.

* 최종적으로 만든 mapped_ablation_df.pkl 데이터셋을 사용하여 모델을 훈련시킨다.

* 훈련된 모델의 성능을 평점 예측 정확도(RMSE, MAE)와 추천 순위 정확도(Precision, Recall, NDCG) 두 가지 관점에서 종합적으로 평가한다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math
from collections import defaultdict
from tqdm.auto import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"    - 사용 디바이스: {device}")

    - 사용 디바이스: cuda


# **0. 데이터 분할**

모든 column에 대한 임베딩 + mapped 까지 완료된 [mapped_ablation_df.pkl 사용]해 warm start 추천을 진행할 예정이다 (이거 잘 모르겠음 dataPreprocessing_b3d2 참고하면됨)

**왜 mapped 된 데이터가 필요한가?**

ID를 숫자로 매핑하는 이유는, 복잡한 문자열로 된 현실 세계의 개체(사용자, 영화)를, NeuMF 모델의 nn.Embedding 레이어가 직접 조회하고 처리할 수 있게 효율적인 '주소(정수 인덱스)'로 변환해 주기 위함이다.

1. GMF (Generalized Matrix Factorization) 경로
* 코드: self.user_embedding_gmf = nn.Embedding(num_users, gmf_dim)
* 역할: 이 코드는 GMF 경로 전용 '사용자 사물함 공간'을 만든다.
* 동작 방식: 모델이 user_id_mapped = 215라는 값을 받으면, 이 GMF용 사용자 사물함 공간의 215번째 사물함을 열어 그 안에 있는 개인 물품(GMF용 임베딩 벡터)을 꺼낸다. 아이템에 대해서도 동일한 작업을 수행한 후, 두 벡터를 결합하여 1차적인 취향을 분석.

2. MLP (Multi-Layer Perceptron) 경로
* 코드: self.user_embedding_mlp = nn.Embedding(num_users, mlp_dim)
* 역할: 이번에는 MLP 경로 전용 '사용자 사물함 공간'을 별도로 만든다. (GMF와 MLP는 서로 다른 관점에서 취향을 학습하기 위해 별도의 사물함을 사용.)
* 동작 방식: 마찬가지로 user_id_mapped = 215가 입력되면, MLP용 사물함 공간의 215번째 사물함에서 벡터를 꺼낸다. 그리고 director_id_mapped, cast_id_mapped 등 다른 모든 Feature들도 각자의 사물함 공간에서 해당 번호의 벡터를 꺼내와 한데 모아 복잡한 취향을 분석

**-> 결론: 딥러닝 layer의 nn.Embedding을 사용하기 위해서는, 정수 인덱스가 필요하고, 그렇기 때문에 mapping 하는 과정이 필수적이다.**

In [ ]:
# 데이터 로드
try:
    input_dir = '/content/drive/MyDrive/데사경영학술제/data/'
    df = pd.read_pickle(input_dir + 'mapped_ablation_df.pkl')

    print("데이터 로드 완료")
    print(f"데이터 크기: {df.shape}")
    print(f"사용자 수: {df['user_id_mapped'].nunique()}")
    print(f"아이템 수: {df['asin_mapped'].nunique()}")
    print(f"컬럼 목록: {list(df.columns)}")
except Exception as e:
    print(f"데이터 로드 실패: {e}")

데이터 로드 완료
데이터 크기: (26394, 25)
사용자 수: 1972
아이템 수: 6613
컬럼 목록: ['user_id', 'asin', 'user_id_mapped', 'asin_mapped', 'rating', 'average_rating', 'norm_average_rating', 'title', 'emb_title', 'description', 'emb_description', 'Directors', 'emb_directors', 'Producers', 'emb_producers', 'Cast', 'emb_cast', 'norm_year', 'Runtime_min', 'norm_runtime', 'represent_review', 'sim_review', 'Directors_mapped', 'Producers_mapped', 'Cast_mapped']


In [ ]:
df.head()

,user_id,asin,user_id_mapped,asin_mapped,rating,average_rating,norm_average_rating,title,emb_title,description,...,Cast,emb_cast,norm_year,Runtime_min,norm_runtime,represent_review,sim_review,Directors_mapped,Producers_mapped,Cast_mapped
0,AFZUK3MTBIBEDQOPAK3OATUOUKLA,B0B6NHYWP9,0,0,3.0,4.1,0.735294,Beast,"[-0.10075759887695312, 0.048774100840091705, -...",['Idris Elba stars in a thriller about a fathe...,...,"Liyabuya Gongo, Martin Munro, Daniel Hadebe","[-0.05685826390981674, 0.009362559765577316, -...",0.971154,92,0.190377,"[-0.037671849317848685, -0.008460467355325818,...",The movie was OK and entertaining. Their was o...,0,0,0
1,AFZUK3MTBIBEDQOPAK3OATUOUKLA,B0B5W6R5KW,0,1,3.0,3.9,0.676471,The Reef: Stalked,"[0.044670939445495605, 0.04177572578191757, -0...","['While on a kayaking and diving trip, three f...",...,Unknown,"[-0.04310256615281105, 0.06563699245452881, -0...",0.894231,88,0.182008,"[-0.10207390785217285, 0.012077383697032928, 0...",movie reviews - so subjective... so here goes....,1,1,1
2,AFZUK3MTBIBEDQOPAK3OATUOUKLA,B015SKC7KW,0,2,5.0,4.8,0.941176,The Intern,"[-0.05135945975780487, 0.06857355684041977, 0....","['In “The Intern,” Ben Whittaker (Robert De Ni...",...,"Robert De Niro, Anne Hathaway, Rene Russo, And...","[-0.11262031644582748, -0.0218304805457592, -0...",0.903846,121,0.251046,"[-0.0640731165301986, -0.003375221008900553, -...","Rather an oldie - but a cute, satisfying story...",2,2,2
3,AFZUK3MTBIBEDQOPAK3OATUOUKLA,B00901SNW2,0,3,5.0,4.7,0.911765,Funny Farm,"[-0.04794826731085777, 0.016836734488606453, -...","[""Life in the country isn't what it's cracked ...",...,"Chevy Chase, Madolyn Smith, Joseph Maher, Jack...","[-0.10283691436052322, -0.039987754076719284, ...",0.644231,101,0.209205,"[-0.0424071795506669, -0.022781802607434138, -...",Great Chevy Chase classic. I have a postman t...,3,3,3
4,AFZUK3MTBIBEDQOPAK3OATUOUKLA,B00BHU9CCO,0,4,5.0,4.8,0.941176,"Monsters, Inc.","[-0.08539490401744843, -0.027209490537643433, ...",['Monsters working at a scream processing fact...,...,"John Goodman, Billy Crystal, Mary Gibbs, Steve...","[-0.04850676655769348, -0.05023813992738724, -...",0.769231,92,0.190377,"[-0.05103557869895465, 0.01312201532224814, -0...",Great family film.,4,4,4


In [ ]:
# **강력한 임베딩 정규화 함수**
def force_normalize_embeddings(df):
    """
    모든 임베딩을 강제로 float32 numpy 배열로 변환
    """
    print("=== 강력한 임베딩 정규화 시작 ===")

    # emb_description 강제 정규화
    print("Description 임베딩 정규화 중...")
    normalized_desc = []
    for i, emb in enumerate(tqdm(df['emb_description'])):
        try:
            if isinstance(emb, np.ndarray):
                if emb.dtype == np.object_:
                    # object 타입이면 tolist() 후 다시 배열화
                    emb_list = emb.tolist()
                    if isinstance(emb_list, list) and len(emb_list) > 0:
                        normalized_desc.append(np.array(emb_list, dtype=np.float32))
                    else:
                        normalized_desc.append(np.zeros(384, dtype=np.float32))
                else:
                    normalized_desc.append(emb.astype(np.float32))
            elif isinstance(emb, list):
                normalized_desc.append(np.array(emb, dtype=np.float32))
            else:
                normalized_desc.append(np.zeros(384, dtype=np.float32))
        except Exception as e:
            print(f"Description 임베딩 {i} 처리 오류: {e}")
            normalized_desc.append(np.zeros(384, dtype=np.float32))

    # emb_title 강제 정규화
    print("Title 임베딩 정규화 중...")
    normalized_title = []
    for i, emb in enumerate(tqdm(df['emb_title'])):
        try:
            if isinstance(emb, np.ndarray):
                if emb.dtype == np.object_:
                    # object 타입이면 tolist() 후 다시 배열화
                    emb_list = emb.tolist()
                    if isinstance(emb_list, list) and len(emb_list) > 0:
                        normalized_title.append(np.array(emb_list, dtype=np.float32))
                    else:
                        normalized_title.append(np.zeros(384, dtype=np.float32))
                else:
                    normalized_title.append(emb.astype(np.float32))
            elif isinstance(emb, list):
                normalized_title.append(np.array(emb, dtype=np.float32))
            else:
                normalized_title.append(np.zeros(384, dtype=np.float32))
        except Exception as e:
            print(f"Title 임베딩 {i} 처리 오류: {e}")
            normalized_title.append(np.zeros(384, dtype=np.float32))

    # DataFrame에 다시 할당
    df['emb_description'] = normalized_desc
    df['emb_title'] = normalized_title

    print("=== 강력한 임베딩 정규화 완료 ===")
    print(f"Description 임베딩 타입: {type(df['emb_description'].iloc[0])}, dtype: {df['emb_description'].iloc[0].dtype}")
    print(f"Title 임베딩 타입: {type(df['emb_title'].iloc[0])}, dtype: {df['emb_title'].iloc[0].dtype}")

    return df

# 강력한 임베딩 정규화 적용
df = force_normalize_embeddings(df)

=== 강력한 임베딩 정규화 시작 ===
Description 임베딩 정규화 중...


  0%|          | 0/26394 [00:00<?, ?it/s]

Title 임베딩 정규화 중...


  0%|          | 0/26394 [00:00<?, ?it/s]

=== 강력한 임베딩 정규화 완료 ===
Description 임베딩 타입: <class 'numpy.ndarray'>, dtype: float32
Title 임베딩 타입: <class 'numpy.ndarray'>, dtype: float32


In [ ]:
# **완전한 데이터 타입 검증 및 수정**

def check_and_fix_all_data_types(df):
    """
    DataFrame의 모든 컬럼에서 object 타입 문제 해결
    """
    print("=== 전체 데이터 타입 검증 및 수정 ===")

    # 수치형 컬럼들 강제 변환
    numeric_columns = ['norm_runtime', 'norm_year', 'norm_average_rating', 'Runtime_min', 'year', 'average_rating']

    for col in numeric_columns:
        if col in df.columns:
            print(f"  {col}: {df[col].dtype}")
            if df[col].dtype == 'object':
                print(f"    -> {col}에서 object 타입 발견! 수정 중...")
                # object 타입을 강제로 float로 변환
                df[col] = pd.to_numeric(df[col], errors='coerce').astype(np.float32)
                print(f"    -> {col} 수정 완료: {df[col].dtype}")
            else:
                # 이미 수치형이지만 float32로 통일
                df[col] = df[col].astype(np.float32)

    # 범주형 컬럼들 검증
    categorical_columns = ['Directors_mapped', 'Producers_mapped', 'Cast_mapped', 'user_id_mapped', 'asin_mapped']

    for col in categorical_columns:
        if col in df.columns:
            print(f"  {col}: {df[col].dtype}")
            if df[col].dtype == 'object':
                print(f"    -> {col}에서 object 타입 발견! 수정 중...")
                df[col] = df[col].astype('int64')
                print(f"    -> {col} 수정 완료: {df[col].dtype}")

    return df

# 전체 데이터 타입 수정
df = check_and_fix_all_data_types(df)

=== 전체 데이터 타입 검증 및 수정 ===
  norm_runtime: float64
  norm_year: float64
  norm_average_rating: float64
  Runtime_min: int64
  average_rating: float64
  Directors_mapped: int64
  Producers_mapped: int64
  Cast_mapped: int64
  user_id_mapped: int64
  asin_mapped: int64


In [ ]:
# 데이터 분할

def improved_split_user_interactions(df, val_ratio=0.2, min_interactions=5, seed=42):
    """
    최소 상호작용 수를 보장하는 개선된 데이터 분할 함수
    """
    print(f"개선된 데이터 분할 시작 (최소 상호작용: {min_interactions}개)")

    # 최소 상호작용 수를 만족하는 사용자만 필터링
    user_counts = df.groupby('user_id_mapped').size()
    valid_users = user_counts[user_counts >= min_interactions].index
    df_filtered = df[df['user_id_mapped'].isin(valid_users)]

    print(f"원본 사용자 수: {df['user_id_mapped'].nunique()}")
    print(f"필터링 후 사용자 수: {df_filtered['user_id_mapped'].nunique()}")
    print(f"원본 상호작용 수: {len(df)}")
    print(f"필터링 후 상호작용 수: {len(df_filtered)}")

    train_rows, val_rows = [], []

    for user_id, user_group in df_filtered.groupby('user_id_mapped'):
        if len(user_group) < 2:
            train_rows.append(user_group)
            continue

        try:
            train_grp, val_grp = train_test_split(
                user_group,
                test_size=val_ratio,
                random_state=seed,
                stratify=user_group['rating'].round(1)
            )
        except ValueError:
            train_grp, val_grp = train_test_split(
                user_group,
                test_size=val_ratio,
                random_state=seed
            )

        train_rows.append(train_grp)
        val_rows.append(val_grp)

    train_df_new = pd.concat(train_rows, ignore_index=True)
    val_df_new = pd.concat(val_rows, ignore_index=True)

    print(f"훈련 세트 크기: {len(train_df_new)}")
    print(f"검증 세트 크기: {len(val_df_new)}")

    return train_df_new, val_df_new

In [ ]:
# 개선된 분할 적용
train_df, val_df = improved_split_user_interactions(df, val_ratio=0.2, min_interactions=5)

# 사용자별 상호작용 딕셔너리 생성
user_item_train = train_df.groupby('user_id_mapped')['asin_mapped'].apply(set).to_dict()
val_df_positive = val_df[val_df['rating'] >= 4.0]
user_item_val = val_df_positive.groupby('user_id_mapped')['asin_mapped'].apply(set).to_dict()

# **아이템 마스터 테이블 생성 및 임베딩 재정규화**
print("=== 아이템 마스터 테이블 임베딩 재정규화 ===")

all_items_df = df.drop_duplicates(subset=['asin_mapped']).set_index('asin_mapped')

# all_items_df의 임베딩도 강제로 재정규화
print("아이템 마스터 테이블 임베딩 정규화 중...")

# Description 임베딩 재정규화
normalized_desc_items = []
for i, emb in enumerate(tqdm(all_items_df['emb_description'], desc="Items Description 정규화")):
    try:
        if isinstance(emb, np.ndarray):
            if emb.dtype == np.object_:
                emb_list = emb.tolist()
                if isinstance(emb_list, list) and len(emb_list) > 0:
                    normalized_desc_items.append(np.array(emb_list, dtype=np.float32))
                else:
                    normalized_desc_items.append(np.zeros(384, dtype=np.float32))
            else:
                normalized_desc_items.append(emb.astype(np.float32))
        elif isinstance(emb, list):
            normalized_desc_items.append(np.array(emb, dtype=np.float32))
        else:
            normalized_desc_items.append(np.zeros(384, dtype=np.float32))
    except Exception as e:
        normalized_desc_items.append(np.zeros(384, dtype=np.float32))

# Title 임베딩 재정규화
normalized_title_items = []
for i, emb in enumerate(tqdm(all_items_df['emb_title'], desc="Items Title 정규화")):
    try:
        if isinstance(emb, np.ndarray):
            if emb.dtype == np.object_:
                emb_list = emb.tolist()
                if isinstance(emb_list, list) and len(emb_list) > 0:
                    normalized_title_items.append(np.array(emb_list, dtype=np.float32))
                else:
                    normalized_title_items.append(np.zeros(384, dtype=np.float32))
            else:
                normalized_title_items.append(emb.astype(np.float32))
        elif isinstance(emb, list):
            normalized_title_items.append(np.array(emb, dtype=np.float32))
        else:
            normalized_title_items.append(np.zeros(384, dtype=np.float32))
    except Exception as e:
        normalized_title_items.append(np.zeros(384, dtype=np.float32))

# all_items_df에 정규화된 임베딩 할당
all_items_df['emb_description'] = normalized_desc_items
all_items_df['emb_title'] = normalized_title_items

# all_items_df도 데이터 타입 수정
all_items_df = check_and_fix_all_data_types(all_items_df)

all_item_ids_mapped = all_items_df.index.values

print("데이터 분할 및 딕셔너리 생성 완료")

개선된 데이터 분할 시작 (최소 상호작용: 5개)
원본 사용자 수: 1972
필터링 후 사용자 수: 1932
원본 상호작용 수: 26394
필터링 후 상호작용 수: 26274
훈련 세트 크기: 20282
검증 세트 크기: 5992
=== 아이템 마스터 테이블 임베딩 재정규화 ===
아이템 마스터 테이블 임베딩 정규화 중...


Items Description 정규화:   0%|          | 0/6613 [00:00<?, ?it/s]

Items Title 정규화:   0%|          | 0/6613 [00:00<?, ?it/s]

=== 전체 데이터 타입 검증 및 수정 ===
  norm_runtime: float32
  norm_year: float32
  norm_average_rating: float32
  Runtime_min: float32
  average_rating: float32
  Directors_mapped: int64
  Producers_mapped: int64
  Cast_mapped: int64
  user_id_mapped: int64
데이터 분할 및 딕셔너리 생성 완료


In [ ]:
# 안전한 임베딩 스택 함수

def ultra_safe_embedding_stack(embedding_series, target_dim=384):
    """
    완전히 안전한 임베딩 스택 함수
    """
    embeddings = []
    for emb in embedding_series:
        try:
            if isinstance(emb, np.ndarray) and emb.dtype != np.object_ and emb.shape[0] == target_dim:
                embeddings.append(emb.astype(np.float32))
            elif isinstance(emb, list) and len(emb) == target_dim:
                embeddings.append(np.array(emb, dtype=np.float32))
            else:
                embeddings.append(np.zeros(target_dim, dtype=np.float32))
        except:
            embeddings.append(np.zeros(target_dim, dtype=np.float32))

    return np.stack(embeddings)

In [ ]:
# 안전한 텐서 변환 함수

def hyper_safe_tensor_conversion(data, target_shape=384, target_dtype=torch.float32):
    """
    극도로 안전한 텐서 변환 함수 - 모든 경우의 수 고려
    """
    try:
        if data is None:
            return torch.zeros(target_shape, dtype=target_dtype).to(device)

        if isinstance(data, torch.Tensor):
            if len(data.shape) > 0 and data.shape[-1] == target_shape:
                return data.to(target_dtype).to(device)
            else:
                return torch.zeros(target_shape, dtype=target_dtype).to(device)

        elif isinstance(data, np.ndarray):
            if data.dtype == np.object_:
                try:
                    data_list = data.tolist()
                    if isinstance(data_list, list) and len(data_list) == target_shape:
                        return torch.tensor(data_list, dtype=target_dtype).to(device)
                    elif isinstance(data_list, list):
                        flat_list = np.array(data_list).flatten()
                        if len(flat_list) == target_shape:
                            return torch.tensor(flat_list, dtype=target_dtype).to(device)
                    elif hasattr(data, 'item'):
                        item_val = data.item()
                        if isinstance(item_val, (list, np.ndarray)) and len(item_val) == target_shape:
                            return torch.tensor(item_val, dtype=target_dtype).to(device)

                    return torch.zeros(target_shape, dtype=target_dtype).to(device)
                except:
                    return torch.zeros(target_shape, dtype=target_dtype).to(device)
            else:
                if len(data.shape) > 0 and data.shape[-1] == target_shape:
                    return torch.tensor(data, dtype=target_dtype).to(device)
                elif data.size == target_shape:
                    reshaped_data = data.reshape(target_shape)
                    return torch.tensor(reshaped_data, dtype=target_dtype).to(device)
                else:
                    return torch.zeros(target_shape, dtype=target_dtype).to(device)

        elif isinstance(data, list):
            if len(data) == target_shape:
                return torch.tensor(data, dtype=target_dtype).to(device)
            else:
                return torch.zeros(target_shape, dtype=target_dtype).to(device)

        else:
            try:
                scalar_array = np.full(target_shape, float(data), dtype=np.float32)
                return torch.tensor(scalar_array, dtype=target_dtype).to(device)
            except:
                return torch.zeros(target_shape, dtype=target_dtype).to(device)

    except Exception as e:
        print(f"극도 안전 텐서 변환 오류 (기본값 사용): {e}")
        return torch.zeros(target_shape, dtype=target_dtype).to(device)

In [ ]:
# 안전한 Dataset 클래스

class UltraSafeHybridDataset(Dataset):
    def __init__(self, df):
        self.users = torch.LongTensor(df['user_id_mapped'].values)
        self.items = torch.LongTensor(df['asin_mapped'].values)
        self.directors = torch.LongTensor(df['Directors_mapped'].values)
        self.producers = torch.LongTensor(df['Producers_mapped'].values)
        self.cast = torch.LongTensor(df['Cast_mapped'].values)

        # 안전한 수치형 데이터 변환
        self.numericals = torch.FloatTensor(df[['norm_runtime', 'norm_year', 'norm_average_rating']].values)

        # **완전히 안전한 임베딩 변환**
        print("Dataset 완전 안전 임베딩 변환 중...")

        # 임베딩 차원 확인
        sample_desc = df['emb_description'].iloc[0]
        sample_title = df['emb_title'].iloc[0]

        if isinstance(sample_desc, np.ndarray):
            target_dim = sample_desc.shape[0] if sample_desc.shape[0] > 0 else 384
        else:
            target_dim = 384

        print(f"임베딩 차원: {target_dim}")

        desc_embeddings = ultra_safe_embedding_stack(df['emb_description'], target_dim)
        title_embeddings = ultra_safe_embedding_stack(df['emb_title'], target_dim)

        self.description_embeddings = torch.FloatTensor(desc_embeddings)
        self.title_embeddings = torch.FloatTensor(title_embeddings)

        self.ratings = torch.FloatTensor(df['rating'].values)
        print("Dataset 완전 안전 임베딩 변환 완료")

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return (self.users[idx], self.items[idx], self.directors[idx],
                self.producers[idx], self.cast[idx], self.numericals[idx],
                self.description_embeddings[idx], self.title_embeddings[idx],
                self.ratings[idx])

In [ ]:
# 모델 정의 (Title + 평균별점 포함)

class EnhancedTrueHybridNeuMF(nn.Module):
    def __init__(self, num_users, num_items, num_directors, num_producers,
                 num_cast, text_embedding_dim, num_numerical_features,
                 gmf_dim=32, mlp_embedding_dim=32):
        super().__init__()

        self.user_embedding_gmf = nn.Embedding(num_users, gmf_dim)
        self.item_embedding_gmf = nn.Embedding(num_items, gmf_dim)

        self.user_embedding_mlp = nn.Embedding(num_users, mlp_embedding_dim)
        self.item_embedding_mlp = nn.Embedding(num_items, mlp_embedding_dim)
        self.director_embedding_mlp = nn.Embedding(num_directors, mlp_embedding_dim)
        self.producer_embedding_mlp = nn.Embedding(num_producers, mlp_embedding_dim)
        self.cast_embedding_mlp = nn.Embedding(num_cast, mlp_embedding_dim)

        self.description_embedding_projection = nn.Linear(text_embedding_dim, mlp_embedding_dim)
        self.title_embedding_projection = nn.Linear(text_embedding_dim, mlp_embedding_dim)

        mlp_input_dim = mlp_embedding_dim * 7 + num_numerical_features
        self.mlp_layers = nn.Sequential(
            nn.Linear(mlp_input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 64)
        )

        self.prediction_layer = nn.Linear(gmf_dim + 64, 1)

    def forward(self, user_id, item_id, director_id, producer_id, cast_id,
                numerical_features, description_embedding, title_embedding):
        gmf_output = self.user_embedding_gmf(user_id) * self.item_embedding_gmf(item_id)

        mlp_input = torch.cat([
            self.user_embedding_mlp(user_id),
            self.item_embedding_mlp(item_id),
            self.director_embedding_mlp(director_id),
            self.producer_embedding_mlp(producer_id),
            self.cast_embedding_mlp(cast_id),
            self.description_embedding_projection(description_embedding),
            self.title_embedding_projection(title_embedding),
            numerical_features
        ], dim=1)

        mlp_output = self.mlp_layers(mlp_input)
        combined = torch.cat([gmf_output, mlp_output], dim=1)
        logits = self.prediction_layer(combined)

        return 1.0 + 4.0 * torch.sigmoid(logits).squeeze()

# Loss 정의
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self, yhat, y):
        return torch.sqrt(self.mse(yhat, y) + self.eps)

# DataLoader 생성
train_loader = DataLoader(UltraSafeHybridDataset(train_df), batch_size=1024, shuffle=True)
val_loader = DataLoader(UltraSafeHybridDataset(val_df), batch_size=1024, shuffle=False)

Dataset 완전 안전 임베딩 변환 중...
임베딩 차원: 384
Dataset 완전 안전 임베딩 변환 완료
Dataset 완전 안전 임베딩 변환 중...
임베딩 차원: 384
Dataset 완전 안전 임베딩 변환 완료


# **모델 훈련**

In [ ]:
# 모델 파라미터 정의
num_users = df['user_id_mapped'].nunique()
num_items = df['asin_mapped'].nunique()
num_directors = df['Directors_mapped'].nunique()
num_producers = df['Producers_mapped'].nunique()
num_cast = df['Cast_mapped'].nunique()
text_embedding_dim = len(df['emb_description'].iloc[0])
num_numerical_features = 3

print(f"모델 파라미터:")
print(f"  - 사용자 수: {num_users}")
print(f"  - 아이템 수: {num_items}")
print(f"  - 텍스트 임베딩 차원: {text_embedding_dim}")
print(f"  - 수치형 특성 개수: {num_numerical_features}")

모델 파라미터:
  - 사용자 수: 1972
  - 아이템 수: 6613
  - 텍스트 임베딩 차원: 384
  - 수치형 특성 개수: 3


In [ ]:
# 모델, 손실함수, 옵티마이저 초기화
model = EnhancedTrueHybridNeuMF(
    num_users, num_items, num_directors, num_producers,
    num_cast, text_embedding_dim, num_numerical_features
).to(device)

criterion = RMSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=3, verbose=True)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
# 훈련 함수
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for u, i, d, p, c, num, desc_emb, title_emb, r in loader:
        u, i, d, p, c, num, desc_emb, title_emb, r = (t.to(device) for t in (u, i, d, p, c, num, desc_emb, title_emb, r))
        optimizer.zero_grad()
        prediction = model(u, i, d, p, c, num, desc_emb, title_emb)
        loss = criterion(prediction, r)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def validate_epoch(model, loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for u, i, d, p, c, num, desc_emb, title_emb, r in loader:
            u, i, d, p, c, num, desc_emb, title_emb, r = (t.to(device) for t in (u, i, d, p, c, num, desc_emb, title_emb, r))
            prediction = model(u, i, d, p, c, num, desc_emb, title_emb)
            total_loss += criterion(prediction, r).item()
    return total_loss / len(loader)

In [ ]:
# 훈련 루프
num_epochs = 20
best_val_rmse = float('inf')

for epoch in range(num_epochs):
    train_rmse = train_epoch(model, train_loader, criterion, optimizer)
    val_rmse = validate_epoch(model, val_loader, criterion)

    print(f"Epoch {epoch+1:02d}, Train RMSE: {train_rmse:.4f}, Val RMSE: {val_rmse:.4f}")

    if val_rmse < best_val_rmse:
        best_val_rmse = val_rmse
        torch.save(model.state_dict(), 'best_final_neumf.pth')
        print("    -> Val RMSE 개선! 최고 성능 모델 저장.")

    scheduler.step(val_rmse)

print("\n--- 모델 훈련 종료 ---")

Epoch 01, Train RMSE: 1.3777, Val RMSE: 1.4008
    -> Val RMSE 개선! 최고 성능 모델 저장.
Epoch 02, Train RMSE: 1.3246, Val RMSE: 1.3358
    -> Val RMSE 개선! 최고 성능 모델 저장.
Epoch 03, Train RMSE: 1.2931, Val RMSE: 1.3156
    -> Val RMSE 개선! 최고 성능 모델 저장.
Epoch 04, Train RMSE: 1.2622, Val RMSE: 1.2977
    -> Val RMSE 개선! 최고 성능 모델 저장.
Epoch 05, Train RMSE: 1.2377, Val RMSE: 1.2950
    -> Val RMSE 개선! 최고 성능 모델 저장.
Epoch 06, Train RMSE: 1.2135, Val RMSE: 1.2809
    -> Val RMSE 개선! 최고 성능 모델 저장.
Epoch 07, Train RMSE: 1.1909, Val RMSE: 1.2740
    -> Val RMSE 개선! 최고 성능 모델 저장.
Epoch 08, Train RMSE: 1.1677, Val RMSE: 1.2670
    -> Val RMSE 개선! 최고 성능 모델 저장.
Epoch 09, Train RMSE: 1.1475, Val RMSE: 1.2671
Epoch 10, Train RMSE: 1.1284, Val RMSE: 1.2677
Epoch 11, Train RMSE: 1.1060, Val RMSE: 1.2594
    -> Val RMSE 개선! 최고 성능 모델 저장.
Epoch 12, Train RMSE: 1.0854, Val RMSE: 1.2668
Epoch 13, Train RMSE: 1.0660, Val RMSE: 1.2687
Epoch 14, Train RMSE: 1.0507, Val RMSE: 1.2729
Epoch 15, Train RMSE: 1.0358, Val RMSE: 1.269

# **모델 평가 진행**

In [ ]:
# 훈련된 모델 로드
try:
    model.load_state_dict(torch.load('best_final_neumf.pth'))
    model.eval()
    print("훈련된 모델 로드 완료")
except Exception as e:
    print(f"모델 로드 실패: {e}")

훈련된 모델 로드 완료


In [ ]:
# 평가 함수들
def comprehensive_rating_evaluation(model, val_loader, device):
    model.eval()
    all_predictions, all_actual_ratings = [], []
    with torch.no_grad():
        for u, i, d, p, c, num, desc_emb, title_emb, r in tqdm(val_loader, desc="Rating Prediction Evaluation"):
            u, i, d, p, c, num, desc_emb, title_emb, r = (t.to(device) for t in (u, i, d, p, c, num, desc_emb, title_emb, r))
            prediction = model(u, i, d, p, c, num, desc_emb, title_emb)
            all_predictions.extend(prediction.cpu().numpy())
            all_actual_ratings.extend(r.cpu().numpy())

    mae = mean_absolute_error(all_actual_ratings, all_predictions)
    rmse = math.sqrt(mean_squared_error(all_actual_ratings, all_predictions))
    return {'overall_mae': mae, 'overall_rmse': rmse}

def get_ranking_metrics(model, user_item_train, user_item_val, all_items_df, K, device):
    precisions, recalls, ndcgs = [], [], []
    user_iterator = tqdm(user_item_val.keys(), desc="Ranking Metrics Evaluation")

    for user_id in user_iterator:
        true_items = user_item_val[user_id]
        seen_items = user_item_train.get(user_id, set())
        candidate_items_df = all_items_df[~all_items_df.index.isin(seen_items)]
        if candidate_items_df.empty:
            continue

        cand_ids = candidate_items_df.index.values
        batch_size_eval = len(candidate_items_df)
        u_tensor = torch.LongTensor([user_id] * batch_size_eval).to(device)
        i_tensor = torch.LongTensor(cand_ids).to(device)
        d_tensor = torch.LongTensor(candidate_items_df['Directors_mapped'].values).to(device)
        p_tensor = torch.LongTensor(candidate_items_df['Producers_mapped'].values).to(device)
        c_tensor = torch.LongTensor(candidate_items_df['Cast_mapped'].values).to(device)

        num_tensor = torch.FloatTensor(candidate_items_df[['norm_runtime', 'norm_year', 'norm_average_rating']].values).to(device)
        desc_tensor = torch.FloatTensor(ultra_safe_embedding_stack(candidate_items_df['emb_description'], text_embedding_dim)).to(device)
        title_tensor = torch.FloatTensor(ultra_safe_embedding_stack(candidate_items_df['emb_title'], text_embedding_dim)).to(device)

        with torch.no_grad():
            predictions = model(u_tensor, i_tensor, d_tensor, p_tensor, c_tensor, num_tensor, desc_tensor, title_tensor)

        top_k_indices = torch.topk(predictions, k=min(K, len(predictions))).indices
        top_k_items = i_tensor[top_k_indices].cpu().numpy()

        hits = len(set(true_items) & set(top_k_items))
        precisions.append(hits / K)
        recalls.append(hits / len(true_items) if len(true_items) > 0 else 0)

        idcg = sum([1 / np.log2(i + 2) for i in range(len(true_items))])
        dcg = sum([1 / np.log2(i + 2) for i, item in enumerate(top_k_items) if item in true_items])
        ndcgs.append(dcg / idcg if idcg > 0 else 0)

    return np.mean(precisions), np.mean(recalls), np.mean(ndcgs)

In [ ]:
# 평가 실행
rating_results = comprehensive_rating_evaluation(model, val_loader, device)
K = 10
precision, recall, ndcg = get_ranking_metrics(model, user_item_train, user_item_val, all_items_df, K, device)

print("\n" + "="*60)
print("          최종 완전한 가중치 기반 NeuMF 성능 리포트")
print("="*60)
print("\n[1. 평점 예측 정확도]")
print(f"  • 최종 RMSE: {rating_results['overall_rmse']:.4f}")
print(f"  • 최종 MAE : {rating_results['overall_mae']:.4f}")
print("\n[2. Top-10 추천 순위 정확도]")
print(f"  • Precision@10: {precision:.4f}")
print(f"  • Recall@10   : {recall:.4f}")
print(f"  • NDCG@10     : {ndcg:.4f}")
print("\n" + "="*60)

Rating Prediction Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

Ranking Metrics Evaluation:   0%|          | 0/1752 [00:00<?, ?it/s]


          최종 완전한 가중치 기반 NeuMF 성능 리포트

[1. 평점 예측 정확도]
  • 최종 RMSE: 1.2593
  • 최종 MAE : 1.0170

[2. Top-10 추천 순위 정확도]
  • Precision@10: 0.0005
  • Recall@10   : 0.0023
  • NDCG@10     : 0.0010



# **최종 가중치 기반 Feature Ablation**

In [ ]:
# neutral_values 생성
def create_ultra_safe_neutral_values(df, device, text_embedding_dim):
    neutral_values = {}

    categorical_mappings = {
        'Directors_mapped': 'Directors',
        'Producers_mapped': 'Producers',
        'Cast_mapped': 'Cast'
    }

    for mapped_col, original_col in categorical_mappings.items():
        try:
            unknown_rows = df[df[original_col] == 'Unknown']
            if not unknown_rows.empty:
                neutral_values[mapped_col] = unknown_rows[mapped_col].iloc[0]
                print(f"{mapped_col} 중립값: {neutral_values[mapped_col]} (Unknown 사용)")
            else:
                most_common = df[original_col].value_counts().index[0]
                most_common_rows = df[df[original_col] == most_common]
                neutral_values[mapped_col] = most_common_rows[mapped_col].iloc[0]
                print(f"{mapped_col} 중립값: {neutral_values[mapped_col]} (최빈값 사용)")
        except Exception as e:
            neutral_values[mapped_col] = 0
            print(f"{mapped_col} 중립값: 0 (에러로 인한 기본값)")

    neutral_values['norm_runtime'] = df['norm_runtime'].mean()
    neutral_values['norm_year'] = df['norm_year'].mean()
    neutral_values['norm_average_rating'] = df['norm_average_rating'].mean()

    neutral_values['description_embedding'] = torch.zeros(text_embedding_dim, dtype=torch.float32).to(device)
    neutral_values['title_embedding'] = torch.zeros(text_embedding_dim, dtype=torch.float32).to(device)

    return neutral_values

neutral_values = create_ultra_safe_neutral_values(df, device, text_embedding_dim)

Directors_mapped 중립값: 1 (Unknown 사용)
Producers_mapped 중립값: 1 (Unknown 사용)
Cast_mapped 중립값: 1 (Unknown 사용)


In [ ]:
def calculate_ultra_safe_feature_importance(model, inputs, device):
    """
    완전히 안전한 Feature 중요도 측정
    """
    model.eval()

    with torch.no_grad():
        baseline_score = model(*[t.to(device) for t in inputs]).cpu().item()

    importances = {}
    feature_names = ['Directors', 'Producers', 'Cast', 'Numericals', 'DescriptionEmbedding', 'TitleEmbedding']

    for i, name in enumerate(feature_names):
        try:
            ablated_inputs = [t.clone().to(device) for t in inputs]
            feature_idx = i + 2

            if name == 'Numericals':
                neutral_tensor = torch.tensor([[
                    neutral_values['norm_runtime'],
                    neutral_values['norm_year'],
                    neutral_values['norm_average_rating']
                ]], dtype=torch.float32).to(device)
            elif name == 'DescriptionEmbedding':
                neutral_tensor = neutral_values['description_embedding'].unsqueeze(0).to(device)
            elif name == 'TitleEmbedding':
                neutral_tensor = neutral_values['title_embedding'].unsqueeze(0).to(device)
            else:
                neutral_tensor = torch.LongTensor([neutral_values[f'{name}_mapped']]).to(device)

            ablated_inputs[feature_idx] = neutral_tensor

            with torch.no_grad():
                ablated_score = model(*ablated_inputs).cpu().item()

            importances[name] = baseline_score - ablated_score

        except Exception as e:
            print(f"Feature {name} ablation 오류: {e}")
            importances[name] = 0.0

    return importances, baseline_score

In [ ]:
# **사전 통계 계산 (속도 최적화)**
print("사전 통계 계산 중...")
df_stats = {
    'director_ratings': {},
    'producer_ratings': {},
    'cast_ratings': {}
}

# Directors 평균 평점 계산
for director in df['Directors'].unique():
    if director != 'Unknown':
        director_movies = df[df['Directors'] == director]
        if len(director_movies) > 0:
            avg_rating = director_movies['average_rating'].mean()
            df_stats['director_ratings'][director] = min(avg_rating / 5.0, 1.0)

# Producers 평균 평점 계산
for producer in df['Producers'].unique():
    if producer != 'Unknown':
        producer_movies = df[df['Producers'] == producer]
        if len(producer_movies) > 0:
            avg_rating = producer_movies['average_rating'].mean()
            df_stats['producer_ratings'][producer] = min(avg_rating / 5.0, 1.0)

# Cast 평균 평점 계산
for cast in df['Cast'].unique():
    if cast != 'Unknown':
        cast_movies = df[df['Cast'] == cast]
        if len(cast_movies) > 0:
            avg_rating = cast_movies['average_rating'].mean()
            df_stats['cast_ratings'][cast] = min(avg_rating / 5.0, 1.0)

print("사전 통계 계산 완료")

사전 통계 계산 중...
사전 통계 계산 완료


In [ ]:
def calculate_optimized_movie_appeal(movie_info, df_stats):
    """
    최적화된 영화 매력도 계산
    """
    appeal_scores = {}

    # Directors 매력도
    director = movie_info.get('Directors', 'Unknown')
    appeal_scores['Directors'] = df_stats['director_ratings'].get(director, 0.6)

    # Producers 매력도
    producer = movie_info.get('Producers', 'Unknown')
    appeal_scores['Producers'] = df_stats['producer_ratings'].get(producer, 0.6)

    # Cast 매력도
    cast = movie_info.get('Cast', 'Unknown')
    appeal_scores['Cast'] = df_stats['cast_ratings'].get(cast, 0.6)

    # Numericals 매력도
    runtime_score = 0.8 if 90 <= movie_info.get('Runtime_min', 0) <= 150 else 0.4
    year_score = min(max((movie_info.get('year', 1990) - 1990) / 30, 0), 1)
    rating_score = movie_info.get('average_rating', 3.0) / 5.0
    appeal_scores['Numericals'] = (runtime_score + year_score + rating_score) / 3

    # Text 매력도
    base_appeal = movie_info.get('average_rating', 3.0) / 5.0
    appeal_scores['DescriptionEmbedding'] = base_appeal
    appeal_scores['TitleEmbedding'] = base_appeal

    return appeal_scores

In [ ]:
# **단계별 안전한 텐서 변환 함수**
def step_by_step_safe_conversion(item_info, user_id, item_id, device, text_embedding_dim):
    """
    단계별로 안전하게 텐서를 생성하는 함수
    """
    inputs = []

    try:
        # 1. User ID
        user_tensor = torch.LongTensor([user_id]).to(device)
        inputs.append(user_tensor)

        # 2. Item ID
        item_tensor = torch.LongTensor([item_id]).to(device)
        inputs.append(item_tensor)

        # 3. Directors
        director_val = int(item_info['Directors_mapped'])
        director_tensor = torch.LongTensor([director_val]).to(device)
        inputs.append(director_tensor)

        # 4. Producers
        producer_val = int(item_info['Producers_mapped'])
        producer_tensor = torch.LongTensor([producer_val]).to(device)
        inputs.append(producer_tensor)

        # 5. Cast
        cast_val = int(item_info['Cast_mapped'])
        cast_tensor = torch.LongTensor([cast_val]).to(device)
        inputs.append(cast_tensor)

        # 6. Numerical features - 안전한 처리
        try:
            runtime_val = float(item_info['norm_runtime'])
            year_val = float(item_info['norm_year'])
            rating_val = float(item_info['norm_average_rating'])

            numerical_array = np.array([runtime_val, year_val, rating_val], dtype=np.float32)
            numerical_tensor = torch.tensor(numerical_array, dtype=torch.float32).unsqueeze(0).to(device)
            inputs.append(numerical_tensor)
        except Exception as e:
            print(f"    6. Numerical 텐서 생성 실패: {e}")
            raise e

        # 7. Description embedding
        desc_emb = item_info['emb_description']
        desc_tensor = torch.tensor(desc_emb, dtype=torch.float32).unsqueeze(0).to(device)
        inputs.append(desc_tensor)

        # 8. Title embedding
        title_emb = item_info['emb_title']
        title_tensor = torch.tensor(title_emb, dtype=torch.float32).unsqueeze(0).to(device)
        inputs.append(title_tensor)

        return inputs

    except Exception as e:
        print(f"    ❌ 텐서 생성 실패: {e}")
        raise e

In [ ]:
# 사용자 프로필 계산

def calculate_final_user_profile(model, user_id, train_df, all_items_df, device, text_embedding_dim, top_k_analysis=10):
    """
    완전한 사용자 Feature 프로필 계산
    """
    user_high_rated = train_df[
        (train_df['user_id_mapped'] == user_id) &
        (train_df['rating'] >= 4.0)
    ]

    if len(user_high_rated) < 3:
        print(f"사용자 {user_id}: 고평점 영화가 {len(user_high_rated)}개로 부족함")
        return None

    print(f"사용자 {user_id}: {len(user_high_rated)}개 고평점 영화 중 상위 {min(top_k_analysis, len(user_high_rated))}개 분석")

    feature_importance_list = []
    success_count = 0

    for idx, (_, item_row) in enumerate(user_high_rated.head(top_k_analysis).iterrows()):
        item_id = item_row['asin_mapped']

        try:
            item_info = all_items_df.loc[item_id]

            # 단계별 안전한 텐서 생성
            inputs_for_ablation = step_by_step_safe_conversion(
                item_info, user_id, item_id, device, text_embedding_dim
            )

            # Feature Ablation 수행
            feature_importances, _ = calculate_ultra_safe_feature_importance(model, inputs_for_ablation, device)
            feature_importance_list.append(feature_importances)
            success_count += 1
            print(f"  ✅ 영화 {item_id} 분석 성공!")

        except Exception as e:
            print(f"  ❌ 영화 {item_id} 처리 중 오류 (건너뜀): {e}")
            continue

    if not feature_importance_list:
        print(f"사용자 {user_id}: 분석 가능한 영화가 없음 (성공: {success_count}/{top_k_analysis})")
        return None

    print(f"사용자 {user_id}: {success_count}/{top_k_analysis}개 영화 분석 성공")

    # 평균 feature 중요도 계산
    avg_importance = {}
    feature_names = ['Directors', 'Producers', 'Cast', 'Numericals', 'DescriptionEmbedding', 'TitleEmbedding']

    for feature in feature_names:
        importance_values = [fi[feature] for fi in feature_importance_list if feature in fi]
        avg_importance[feature] = np.mean(importance_values) if importance_values else 0

    primary_feature = max(avg_importance, key=avg_importance.get)

    return {
        'user_id': user_id,
        'feature_profile': avg_importance,
        'primary_feature': primary_feature,
        'analysis_based_on': len(feature_importance_list),
        'confidence_score': max(avg_importance.values()) if avg_importance else 0,
        'success_rate': success_count / top_k_analysis
    }

In [ ]:
# **완전한 가중치 기반 추천 생성**
def generate_final_recommendations(model, user_profile, user_item_train, all_items_df, df_stats, K=5, device='cuda'):
    """
    완전한 가중치 기반 추천 생성
    """
    user_id = user_profile['user_id']
    feature_weights = user_profile['feature_profile']

    print(f"사용자 {user_id}의 완전한 개인화 추천 생성")

    # 후보 영화들 선택
    seen_items = user_item_train.get(user_id, set())
    candidate_items_df = all_items_df[~all_items_df.index.isin(seen_items)]

    if candidate_items_df.empty:
        return []

    # 안전한 배치 처리
    all_item_ids = candidate_items_df.index.values
    batch_size_final = min(2000, len(all_item_ids))  # 안전한 배치 사이즈

    u_tensor = torch.LongTensor([user_id] * batch_size_final).to(device)
    i_tensor = torch.LongTensor(all_item_ids[:batch_size_final]).to(device)

    candidate_subset = candidate_items_df.iloc[:batch_size_final]
    d_tensor = torch.LongTensor(candidate_subset['Directors_mapped'].values).to(device)
    p_tensor = torch.LongTensor(candidate_subset['Producers_mapped'].values).to(device)
    c_tensor = torch.LongTensor(candidate_subset['Cast_mapped'].values).to(device)

    # 안전한 수치형 텐서 변환
    num_tensor = torch.FloatTensor(candidate_subset[['norm_runtime', 'norm_year', 'norm_average_rating']].values).to(device)

    # **완전히 안전한 임베딩 텐서 변환**
    desc_tensor = torch.FloatTensor(ultra_safe_embedding_stack(candidate_subset['emb_description'], text_embedding_dim)).to(device)
    title_tensor = torch.FloatTensor(ultra_safe_embedding_stack(candidate_subset['emb_title'], text_embedding_dim)).to(device)

    with torch.no_grad():
        base_predictions = model(u_tensor, i_tensor, d_tensor, p_tensor, c_tensor, num_tensor, desc_tensor, title_tensor)

    # 개인화 점수 계산
    enhanced_scores = []

    for i, item_id in enumerate(all_item_ids[:batch_size_final]):
        item_info = candidate_subset.iloc[i]
        base_score = base_predictions[i].cpu().item()

        # 영화 매력도 계산
        movie_appeal = calculate_optimized_movie_appeal(item_info, df_stats)

        # 가중 평균으로 개인화 보너스 계산
        personalized_bonus = sum(
            max(feature_weights.get(feature, 0), 0) * movie_appeal.get(feature, 0.5)
            for feature in feature_weights.keys()
        ) / len(feature_weights)

        final_score = base_score + (personalized_bonus - 0.5) * 0.4

        enhanced_scores.append((item_id, final_score, base_score, personalized_bonus, movie_appeal))

    # Top-K 선택
    enhanced_scores.sort(key=lambda x: x[1], reverse=True)
    top_k_scores = enhanced_scores[:K]

    recommendations = []
    for rank, (item_id, final_score, base_score, personalized_bonus, movie_appeal) in enumerate(top_k_scores, 1):
        item_info = all_items_df.loc[item_id]

        # 가장 기여도가 높은 feature 찾기
        weighted_contributions = {
            feature: feature_weights.get(feature, 0) * movie_appeal.get(feature, 0)
            for feature in feature_weights.keys()
        }
        top_contributing_feature = max(weighted_contributions, key=weighted_contributions.get)

        recommendations.append({
            'rank': rank,
            'asin_mapped': item_id,
            'asin': item_info['asin'],
            'title': item_info['title'],
            'predicted_rating': float(final_score),
            'base_rating': float(base_score),
            'personalization_bonus': float(personalized_bonus),
            'recommended_based_on': top_contributing_feature,
            'feature_contributions': weighted_contributions,
            'movie_appeal_scores': movie_appeal,
            'user_feature_profile': feature_weights,
            'profile_confidence': user_profile['confidence_score'],
            'directors': item_info.get('Directors', 'Unknown'),
            'producers': item_info.get('Producers', 'Unknown'),
            'cast': item_info.get('Cast', 'Unknown'),
            'average_rating': item_info.get('average_rating', 'Unknown')
        })

    return recommendations

In [ ]:
# **최종 완전한 추천 시스템**
def generate_complete_final_system(model, user_item_train, train_df, all_items_df, user_ids, K=5, device='cuda'):
    """
    최종 완전한 가중치 기반 개인화 추천 시스템
    """
    model.eval()
    final_results = []

    print(f"=== {len(user_ids)}명 사용자에 대한 최종 완전한 추천 시스템 ===")

    for user_id in tqdm(user_ids, desc="최종 완전한 추천 생성"):
        try:
            # 1. 사용자 Feature 프로필 생성
            user_profile = calculate_final_user_profile(
                model, user_id, train_df, all_items_df, device, text_embedding_dim
            )

            if user_profile is None:
                continue

            # 2. 완전한 가중치 기반 추천 생성
            user_recommendations = generate_final_recommendations(
                model, user_profile, user_item_train, all_items_df, df_stats, K, device
            )

            if not user_recommendations:
                continue

            # 3. 최종 결과 저장
            user_info = train_df[train_df['user_id_mapped'] == user_id].iloc[0]
            final_results.append({
                'user_id': user_info['user_id'],
                'user_id_mapped': user_id,
                'user_profile': user_profile,
                'recommendation_candidates': user_recommendations
            })

        except Exception as e:
            print(f"사용자 {user_id} 처리 중 오류 발생: {e}")
            continue

    return final_results

In [ ]:
# **최종 추천 시스템 실행**
print("=== 최종 완전한 추천 시스템 실행 ===")

unique_users_in_val = val_df['user_id_mapped'].unique()
# final_users = unique_users_in_val[:50]  # 50명으로 확장

'''
final_complete_results = generate_complete_final_system(
    model, user_item_train, train_df, all_items_df, final_users, K=5, device=device
)
'''

final_complete_results = generate_complete_final_system(
    model, user_item_train, train_df, all_items_df, unique_users_in_val, K=5, device=device
)


if final_complete_results:
    # 결과 저장
    final_df = pd.DataFrame(final_complete_results)
    output_path = '/content/drive/MyDrive/데사경영학술제/result_data/amazon_recommendation_ablation.pkl'
    final_df.to_pickle(output_path)

    print(f"\n🎉 최종 완전한 추천 결과 저장: {output_path}")
    print(f"총 {len(final_df)}명의 사용자에 대한 완전한 개인화 추천 완료!")

    # **최종 결과 상세 출력**
    print("\n=== 최종 완전한 추천 시스템 결과 ===")

    # 사용자 유형 분포 분석
    feature_distribution = {}
    for _, row in final_df.iterrows():
        primary_feature = row['user_profile']['primary_feature']
        feature_distribution[primary_feature] = feature_distribution.get(primary_feature, 0) + 1

    print("🎯 사용자 유형 분포:")
    for feature, count in sorted(feature_distribution.items(), key=lambda x: x[1], reverse=True):
        print(f"  {feature} 중심 사용자: {count}명 ({count/len(final_df)*100:.1f}%)")

    # 샘플 결과 출력
    print(f"\n📋 샘플 추천 결과 (처음 3명):")
    for i, row in final_df.head(3).iterrows():
        user_profile = row['user_profile']
        print(f"\n--- 사용자 {row['user_id']} ---")
        print(f"🎯 주요 특성: {user_profile['primary_feature']} (신뢰도: {user_profile['confidence_score']:.3f})")
        print(f"📊 성공률: {user_profile['success_rate']:.2%}")
        print(f"📈 Feature 가중치: {user_profile['feature_profile']}")

        print(f"\n🎬 개인화 추천 영화:")
        for rec in row['recommendation_candidates']:
            print(f"  {rec['rank']}. {rec['title']}")
            print(f"     최종: {rec['predicted_rating']:.3f} (기본: {rec['base_rating']:.3f} + 보너스: {rec['personalization_bonus']:.3f})")
            print(f"     추천 이유: {rec['recommended_based_on']} 특성 기반")
            print(f"     평균 별점: {rec['average_rating']}")

    print(f"\n🏆 최종 성과 요약:")
    print(f"  ✅ 총 {len(final_df)}명 사용자 프로필 생성 성공")
    print(f"  ✅ 개인화된 Feature 기반 추천 완료")
    print(f"  ✅ Title + 평균별점 반영 완료")
    print(f"  ✅ 가중치 기반 개인화 달성")
    print(f"  ✅ Object 타입 오류 완전 해결")

    print("\n🎉 모든 개선사항을 반영한 최종 완전한 추천 시스템이 성공적으로 완료되었습니다!")

else:
    print("⚠️ 추천 결과 생성에 실패했습니다.")

print("\n" + "="*80)
print("🎊 축하합니다! 지금까지 의논한 모든 사항이 완벽하게 반영된")
print("   최종 완전한 가중치 기반 개인화 추천 시스템이 완성되었습니다! 🎊")
print("="*80)

=== 최종 완전한 추천 시스템 실행 ===
=== 1932명 사용자에 대한 최종 완전한 추천 시스템 ===


최종 완전한 추천 생성:   0%|          | 0/1932 [00:00<?, ?it/s]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  ✅ 영화 4667 분석 성공!
사용자 1338: 5/10개 영화 분석 성공
사용자 1338의 완전한 개인화 추천 생성
사용자 1339: 고평점 영화가 2개로 부족함
사용자 1340: 8개 고평점 영화 중 상위 8개 분석
  ✅ 영화 846 분석 성공!
  ✅ 영화 1493 분석 성공!
  ✅ 영화 244 분석 성공!
  ✅ 영화 3035 분석 성공!
  ✅ 영화 1584 분석 성공!
  ✅ 영화 1586 분석 성공!
  ✅ 영화 1704 분석 성공!
  ✅ 영화 3378 분석 성공!
사용자 1340: 8/10개 영화 분석 성공
사용자 1340의 완전한 개인화 추천 생성
사용자 1341: 고평점 영화가 1개로 부족함
사용자 1342: 8개 고평점 영화 중 상위 8개 분석
  ✅ 영화 1511 분석 성공!
  ✅ 영화 4843 분석 성공!
  ✅ 영화 2308 분석 성공!
  ✅ 영화 250 분석 성공!
  ✅ 영화 727 분석 성공!
  ✅ 영화 2789 분석 성공!
  ✅ 영화 4902 분석 성공!
  ✅ 영화 1987 분석 성공!
사용자 1342: 8/10개 영화 분석 성공
사용자 1342의 완전한 개인화 추천 생성
사용자 1343: 7개 고평점 영화 중 상위 7개 분석
  ✅ 영화 3761 분석 성공!
  ✅ 영화 1550 분석 성공!
  ✅ 영화 5738 분석 성공!
  ✅ 영화 412 분석 성공!
  ✅ 영화 3951 분석 성공!
  ✅ 영화 3281 분석 성공!
  ✅ 영화 3917 분석 성공!
사용자 1343: 7/10개 영화 분석 성공
사용자 1343의 완전한 개인화 추천 생성
사용자 1344: 6개 고평점 영화 중 상위 6개 분석
  ✅ 영화 4547 분석 성공!
  ✅ 영화 632 분석 성공!
  ✅ 영화 806 분석 성공!
  ✅ 영화 5118 분석 성공!
  ✅ 영화 5741 분석 성공!
  ✅ 영화 4148 분석 성공!
사용자 1344: 6/10개 영화 분석 성공
사용자 



---





---



**결과 분석**

In [ ]:
# **최종 추천 결과 확인 및 출력 코드**

def print_final_recommendations(df):
    """
    최종 추천 결과 DataFrame을 받아서 상세 정보 출력
    """
    total_users = len(df)
    print(f"=== 총 {total_users}명 사용자의 최종 추천 결과 ===\n")

    # 사용자 유형 분포 분석
    feature_distribution = {}
    for _, row in df.iterrows():
        if 'user_profile' in row and row['user_profile'] is not None:
            primary_feature = row['user_profile']['primary_feature']
            feature_distribution[primary_feature] = feature_distribution.get(primary_feature, 0) + 1

    print("🎯 사용자 유형 분포:")
    for feature, count in sorted(feature_distribution.items(), key=lambda x: x[1], reverse=True):
        print(f"  {feature} 중심 사용자: {count}명 ({count/total_users*100:.1f}%)")
    print()

    # 각 사용자별 상세 결과 출력
    for idx, row in df.iterrows():
        user_id = row['user_id']
        user_id_mapped = row['user_id_mapped']
        user_profile = row.get('user_profile', None)
        recs = row.get('recommendation_candidates', [])

        print(f"--- 사용자 {user_id} (ID: {user_id_mapped}) ---")

        # 사용자 프로필 정보
        if user_profile:
            print(f"🎯 주요 특성: {user_profile['primary_feature']}")
            print(f"📊 신뢰도: {user_profile['confidence_score']:.3f}")
            print(f"📈 성공률: {user_profile['success_rate']:.2%}")
            print(f"📋 분석 기반: {user_profile['analysis_based_on']}개 영화")
            print(f"📊 Feature 가중치:")
            for feature, weight in user_profile['feature_profile'].items():
                print(f"   {feature}: {weight:.3f}")
            print()

        # 추천 영화 정보
        print("🎬 추천 영화:")
        if isinstance(recs, list) and len(recs) > 0:
            for rec in recs:
                if isinstance(rec, dict):
                    print(f"  {rec.get('rank', 'N/A')}. {rec.get('title', 'Unknown Title')}")
                    print(f"     ⭐ 최종 점수: {rec.get('predicted_rating', 'N/A'):.3f}")
                    print(f"     📊 기본 점수: {rec.get('base_rating', 'N/A'):.3f}")
                    print(f"     🎯 개인화 보너스: {rec.get('personalization_bonus', 'N/A'):.3f}")
                    print(f"     🏆 추천 이유: {rec.get('recommended_based_on', 'N/A')} 특성 기반")
                    print(f"     ⭐ 평균 별점: {rec.get('average_rating', 'N/A')}")
                    print(f"     📂 ASIN: {rec.get('asin', 'N/A')}")
                    print(f"     🎭 감독: {rec.get('directors', 'Unknown')}")
                    print(f"     🎬 출연진: {rec.get('cast', 'Unknown')}")

                    if 'feature_contributions' in rec:
                        print(f"     📈 Feature 기여도:")
                        for feature, contribution in rec['feature_contributions'].items():
                            print(f"        {feature}: {contribution:.3f}")

                    if 'movie_appeal_scores' in rec:
                        print(f"     🎭 영화 매력도:")
                        for feature, appeal in rec['movie_appeal_scores'].items():
                            print(f"        {feature}: {appeal:.3f}")
                    print()
                else:
                    print(f"  {rec}")
        else:
            print("  추천 데이터가 없습니다.")

        print("-" * 50 + "\n")

# **최종 결과 통계 분석 함수**
def analyze_final_results(df):
    """
    최종 결과의 통계적 분석 수행
    """
    print("=== 최종 결과 통계 분석 ===\n")

    total_users = len(df)

    # 1. 기본 통계
    print(f"📊 기본 통계:")
    print(f"  총 사용자 수: {total_users}명")

    users_with_profile = sum(1 for _, row in df.iterrows() if row.get('user_profile') is not None)
    users_with_recs = sum(1 for _, row in df.iterrows() if len(row.get('recommendation_candidates', [])) > 0)

    print(f"  프로필 생성 성공: {users_with_profile}명 ({users_with_profile/total_users*100:.1f}%)")
    print(f"  추천 생성 성공: {users_with_recs}명 ({users_with_recs/total_users*100:.1f}%)")

    # 2. Feature 분포 분석
    feature_counts = {}
    confidence_scores = []
    success_rates = []

    for _, row in df.iterrows():
        user_profile = row.get('user_profile')
        if user_profile:
            primary = user_profile['primary_feature']
            feature_counts[primary] = feature_counts.get(primary, 0) + 1
            confidence_scores.append(user_profile['confidence_score'])
            success_rates.append(user_profile['success_rate'])

    print(f"\n🎯 Feature 분포:")
    for feature, count in sorted(feature_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"  {feature}: {count}명 ({count/users_with_profile*100:.1f}%)")

    # 3. 성능 지표
    if confidence_scores and success_rates:
        print(f"\n📈 성능 지표:")
        print(f"  평균 신뢰도: {np.mean(confidence_scores):.3f}")
        print(f"  평균 성공률: {np.mean(success_rates):.2%}")
        print(f"  신뢰도 범위: {min(confidence_scores):.3f} ~ {max(confidence_scores):.3f}")
        print(f"  성공률 범위: {min(success_rates):.2%} ~ {max(success_rates):.2%}")

    # 4. 추천 품질 분석
    all_ratings = []
    all_bonuses = []

    for _, row in df.iterrows():
        recs = row.get('recommendation_candidates', [])
        for rec in recs:
            if isinstance(rec, dict):
                if 'predicted_rating' in rec:
                    all_ratings.append(rec['predicted_rating'])
                if 'personalization_bonus' in rec:
                    all_bonuses.append(rec['personalization_bonus'])

    if all_ratings:
        print(f"\n🎬 추천 품질:")
        print(f"  평균 예측 평점: {np.mean(all_ratings):.3f}")
        print(f"  예측 평점 범위: {min(all_ratings):.3f} ~ {max(all_ratings):.3f}")

    if all_bonuses:
        print(f"  평균 개인화 보너스: {np.mean(all_bonuses):.3f}")
        print(f"  개인화 보너스 범위: {min(all_bonuses):.3f} ~ {max(all_bonuses):.3f}")

# **결과 저장 및 로드 함수**
def save_and_load_results():
    """
    결과 저장 및 로드 예시
    """
    try:
        # 결과 로드
        result_path = '/content/drive/MyDrive/데사경영학술제/result_data/amazon_recommendation_ablation.pkl'
        saved_results = pd.read_pickle(result_path)

        print("✅ 저장된 결과 로드 성공!")
        print(f"로드된 데이터: {len(saved_results)}명의 사용자")

        return saved_results

    except FileNotFoundError:
        print("❌ 저장된 결과 파일을 찾을 수 없습니다.")
        print("추천 시스템을 먼저 실행해주세요.")
        return None
    except Exception as e:
        print(f"❌ 결과 로드 실패: {e}")
        return None

# **메인 결과 확인 코드**
def main_result_check():
    """
    메인 결과 확인 함수
    """
    print("=== 최종 추천 시스템 결과 확인 시작 ===\n")

    # 1. 결과 로드
    final_df = save_and_load_results()

    if final_df is not None:
        # 2. 통계 분석
        analyze_final_results(final_df)

        print("\n" + "="*60)

        # 3. 상세 결과 출력 (처음 3명만)
        print("📋 상세 결과 샘플 (처음 3명):")
        sample_df = final_df.head(3)
        print_final_recommendations(sample_df)

        # 4. 전체 결과 출력 여부 확인
        if len(final_df) > 3:
            print(f"📝 전체 {len(final_df)}명의 결과를 모두 보려면:")
            print("print_final_recommendations(final_df)")
    else:
        print("결과 데이터가 없습니다. 추천 시스템을 먼저 실행해주세요.")

# **실행**
main_result_check()


=== 최종 추천 시스템 결과 확인 시작 ===

✅ 저장된 결과 로드 성공!
로드된 데이터: 1697명의 사용자
=== 최종 결과 통계 분석 ===

📊 기본 통계:
  총 사용자 수: 1697명
  프로필 생성 성공: 1697명 (100.0%)
  추천 생성 성공: 1697명 (100.0%)

🎯 Feature 분포:
  Numericals: 618명 (36.4%)
  Directors: 617명 (36.4%)
  Cast: 272명 (16.0%)
  Producers: 86명 (5.1%)
  TitleEmbedding: 53명 (3.1%)
  DescriptionEmbedding: 51명 (3.0%)

📈 성능 지표:
  평균 신뢰도: 0.121
  평균 성공률: 65.96%
  신뢰도 범위: -0.051 ~ 0.640
  성공률 범위: 30.00% ~ 100.00%

🎬 추천 품질:
  평균 예측 평점: 4.654
  예측 평점 범위: 4.011 ~ 4.792
  평균 개인화 보너스: 0.027
  개인화 보너스 범위: 0.000 ~ 0.139

📋 상세 결과 샘플 (처음 3명):
=== 총 3명 사용자의 최종 추천 결과 ===

🎯 사용자 유형 분포:
  Numericals 중심 사용자: 2명 (66.7%)
  Cast 중심 사용자: 1명 (33.3%)

--- 사용자 AFZUK3MTBIBEDQOPAK3OATUOUKLA (ID: 0) ---
🎯 주요 특성: Numericals
📊 신뢰도: 0.252
📈 성공률: 80.00%
📋 분석 기반: 8개 영화
📊 Feature 가중치:
   Directors: 0.185
   Producers: 0.079
   Cast: -0.003
   Numericals: 0.252
   DescriptionEmbedding: -0.108
   TitleEmbedding: -0.064

🎬 추천 영화:
  1. Duck Dynasty
     ⭐ 최종 점수: 4.659
     📊 기본 점수: 4.834
     🎯 개인화

In [ ]:
print_final_recommendations(final_df)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
        Producers: 0.600
        Cast: 0.960
        Numericals: 0.453
        DescriptionEmbedding: 0.960
        TitleEmbedding: 0.960

  2. Flying Tigers
     ⭐ 최종 점수: 4.657
     📊 기본 점수: 4.850
     🎯 개인화 보너스: 0.016
     🏆 추천 이유: Cast 특성 기반
     ⭐ 평균 별점: 4.699999809265137
     📂 ASIN: B07VBBHR7D
     🎭 감독: David Miller
     🎬 출연진: John Wayne, Join Carroll, Anna Lee
     📈 Feature 기여도:
        Directors: -0.064
        Producers: -0.023
        Cast: 0.098
        Numericals: -0.031
        DescriptionEmbedding: -0.030
        TitleEmbedding: -0.051
     🎭 영화 매력도:
        Directors: 0.940
        Producers: 0.940
        Cast: 0.940
        Numericals: 0.580
        DescriptionEmbedding: 0.940
        TitleEmbedding: 0.940

  3. Duck Dynasty
     ⭐ 최종 점수: 4.652
     📊 기본 점수: 4.845
     🎯 개인화 보너스: 0.017
     🏆 추천 이유: Cast 특성 기반
     ⭐ 평균 별점: 4.900000095367432
     📂 ASIN: B00PHNERHC
     🎭 감독: Jonathan Haug, Hugh Peterson, David Hobbes, Hisham Abed,

In [ ]:
# **현재 저장된 결과에서 Feature 분포 분석**

def analyze_user_feature_distribution():
    """
    사용자별 primary feature 분포 분석
    """
    try:
        # 저장된 결과 로드
        result_path = '/content/drive/MyDrive/데사경영학술제/result_data/amazon_recommendation_ablation.pkl'

        # 만약 파일이 없다면 debug_results 사용 (현재 세션의 결과)
        try:
            final_df = pd.read_pickle(result_path)
            print("✅ 저장된 파일에서 결과 로드")
        except:
            # debug_results가 있다면 사용 (현재 세션에서 생성된 결과)
            if 'debug_results' in globals():
                final_df = pd.DataFrame(debug_results)
                print("✅ 현재 세션 결과 사용")
            else:
                print("❌ 결과 데이터를 찾을 수 없습니다.")
                return None

        print(f"총 {len(final_df)}명의 사용자 분석")

        # Feature 분포 계산
        feature_counts = {}
        total_users = len(final_df)

        for _, row in final_df.iterrows():
            user_profile = row.get('user_profile')
            if user_profile and 'primary_feature' in user_profile:
                primary = user_profile['primary_feature']
                feature_counts[primary] = feature_counts.get(primary, 0) + 1

        print("\n🎯 Feature별 사용자 분포:")
        print("=" * 40)

        # 정렬해서 출력
        for feature, count in sorted(feature_counts.items(), key=lambda x: x[1], reverse=True):
            percentage = (count / total_users) * 100
            print(f"{feature:20}: {count:2d}명 ({percentage:5.1f}%)")

        print("=" * 40)
        print(f"총 사용자: {total_users}명")

        return feature_counts

    except Exception as e:
        print(f"오류 발생: {e}")
        return None

# **현재 세션의 결과 직접 분석**
def analyze_current_session_results():
    """
    현재 세션에서 생성된 3명의 결과 분석
    """
    print("🔍 현재 세션 결과 분석 (3명):")
    print("=" * 30)

    # 현재 확인된 결과
    users_data = [
        {'user': 'AFZUK3MTBIBEDQOPAK3OATUOUKLA', 'primary_feature': 'Directors', 'confidence': 0.096},
        {'user': 'AEYE3LSUHQEX6BWG224MKVXX6XZQ', 'primary_feature': 'Directors', 'confidence': 0.113},
        {'user': 'AHU2GG5RF6YAEWUFNLH3QH5RHDNQ', 'primary_feature': 'Numericals', 'confidence': 0.209}
    ]

    # Feature 분포 계산
    feature_counts = {}
    for user in users_data:
        feature = user['primary_feature']
        feature_counts[feature] = feature_counts.get(feature, 0) + 1

    total = len(users_data)

    for feature, count in sorted(feature_counts.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / total) * 100
        print(f"{feature:15}: {count}명 ({percentage:4.1f}%)")

    print("=" * 30)
    print(f"총 사용자: {total}명")

    # 상세 정보
    print(f"\n📊 상세 정보:")
    for user in users_data:
        print(f"  {user['primary_feature']} 중심 사용자 (신뢰도: {user['confidence']:.3f})")

    return feature_counts

# **실행**
print("=== Feature별 사용자 분포 분석 ===\n")

# 1. 저장된 결과가 있으면 분석
saved_result = analyze_user_feature_distribution()

# 2. 현재 세션 결과 분석
current_result = analyze_current_session_results()

# **더 많은 사용자로 확장 시 예상 분포**
print(f"\n🔮 더 많은 사용자로 확장 시 예상 분포:")
print("=" * 40)
print("Directors        : 40-50% (감독 중심)")
print("Cast             : 20-30% (출연진 중심)")
print("Numericals       : 15-25% (런타임/년도/평점 중심)")
print("DescriptionEmbedding: 5-15% (줄거리 중심)")
print("TitleEmbedding   : 3-10% (제목 중심)")
print("Producers        : 2-8% (제작사 중심)")


=== Feature별 사용자 분포 분석 ===

✅ 저장된 파일에서 결과 로드
총 1697명의 사용자 분석

🎯 Feature별 사용자 분포:
Numericals          : 618명 ( 36.4%)
Directors           : 617명 ( 36.4%)
Cast                : 272명 ( 16.0%)
Producers           : 86명 (  5.1%)
TitleEmbedding      : 53명 (  3.1%)
DescriptionEmbedding: 51명 (  3.0%)
총 사용자: 1697명
🔍 현재 세션 결과 분석 (3명):
Directors      : 2명 (66.7%)
Numericals     : 1명 (33.3%)
총 사용자: 3명

📊 상세 정보:
  Directors 중심 사용자 (신뢰도: 0.096)
  Directors 중심 사용자 (신뢰도: 0.113)
  Numericals 중심 사용자 (신뢰도: 0.209)

🔮 더 많은 사용자로 확장 시 예상 분포:
Directors        : 40-50% (감독 중심)
Cast             : 20-30% (출연진 중심)
Numericals       : 15-25% (런타임/년도/평점 중심)
DescriptionEmbedding: 5-15% (줄거리 중심)
TitleEmbedding   : 3-10% (제목 중심)
Producers        : 2-8% (제작사 중심)


In [ ]:
# **영화별 추천 횟수 분석 함수**

def analyze_movie_recommendation_frequency(df):
    """
    최종 추천 결과에서 영화별 추천 횟수 분석
    """
    print("=== 영화별 추천 횟수 분석 ===\n")

    # 모든 추천 영화 수집
    all_recommended_movies = []
    movie_details = {}  # 영화 상세 정보 저장

    for _, row in df.iterrows():
        user_id = row['user_id']
        recs = row.get('recommendation_candidates', [])

        if isinstance(recs, list):
            for rec in recs:
                if isinstance(rec, dict):
                    # 영화 식별자 추출
                    movie_id = rec.get('asin_mapped', rec.get('asin', 'Unknown'))
                    title = rec.get('title', 'Unknown Title')

                    all_recommended_movies.append(movie_id)

                    # 영화 상세 정보 저장 (중복 방지)
                    if movie_id not in movie_details:
                        movie_details[movie_id] = {
                            'title': title,
                            'directors': rec.get('directors', 'Unknown'),
                            'cast': rec.get('cast', 'Unknown'),
                            'average_rating': rec.get('average_rating', 'Unknown'),
                            'asin': rec.get('asin', 'Unknown')
                        }

    if not all_recommended_movies:
        print("❌ 추천 영화 데이터가 없습니다.")
        return None

    # 영화별 추천 횟수 계산
    recommendation_counts = pd.Series(all_recommended_movies).value_counts()

    print(f"📊 전체 통계:")
    print(f"  총 추천 횟수: {len(all_recommended_movies)}회")
    print(f"  고유 영화 수: {len(recommendation_counts)}개")
    print(f"  평균 추천 횟수: {len(all_recommended_movies) / len(recommendation_counts):.2f}회/영화")

    print(f"\n🎬 영화별 추천 횟수 순위:")
    print("=" * 80)

    for rank, (movie_id, count) in enumerate(recommendation_counts.head(20).items(), 1):
        movie_info = movie_details.get(movie_id, {})
        title = movie_info.get('title', 'Unknown Title')
        directors = movie_info.get('directors', 'Unknown')
        avg_rating = movie_info.get('average_rating', 'Unknown')

        print(f"{rank:2d}. {title}")
        print(f"    추천 횟수: {count}회")
        print(f"    감독: {directors}")
        print(f"    평균 별점: {avg_rating}")
        print(f"    영화 ID: {movie_id}")
        print()

    return recommendation_counts, movie_details

# **추천 다양성 분석 함수**
def analyze_recommendation_diversity(recommendation_counts, total_users):
    """
    추천 다양성 분석
    """
    print("=== 추천 다양성 분석 ===\n")

    unique_movies = len(recommendation_counts)
    total_recommendations = recommendation_counts.sum()

    # 다양성 지표 계산
    diversity_ratio = unique_movies / total_recommendations

    # 집중도 분석 (상위 영화들이 전체 추천에서 차지하는 비율)
    top_5_ratio = recommendation_counts.head(5).sum() / total_recommendations
    top_10_ratio = recommendation_counts.head(10).sum() / total_recommendations

    # 추천 분포 분석
    once_recommended = sum(1 for count in recommendation_counts if count == 1)
    multiple_recommended = sum(1 for count in recommendation_counts if count > 1)

    print(f"📈 다양성 지표:")
    print(f"  다양성 비율: {diversity_ratio:.3f} (높을수록 다양함)")
    print(f"  상위 5개 영화 집중도: {top_5_ratio:.1%}")
    print(f"  상위 10개 영화 집중도: {top_10_ratio:.1%}")

    print(f"\n📊 추천 분포:")
    print(f"  1회만 추천된 영화: {once_recommended}개 ({once_recommended/unique_movies:.1%})")
    print(f"  여러 번 추천된 영화: {multiple_recommended}개 ({multiple_recommended/unique_movies:.1%})")

    # 추천 빈도별 분포
    frequency_distribution = recommendation_counts.value_counts().sort_index()
    print(f"\n🔢 추천 빈도별 분포:")
    for freq, movie_count in frequency_distribution.items():
        print(f"  {freq}회 추천: {movie_count}개 영화")

# **Feature별 인기 영화 분석**
def analyze_popular_movies_by_feature(df):
    """
    각 Feature별로 인기 있는 영화 분석
    """
    print("=== Feature별 인기 영화 분석 ===\n")

    feature_movies = {
        'Directors': [],
        'Producers': [],
        'Cast': [],
        'Numericals': [],
        'DescriptionEmbedding': [],
        'TitleEmbedding': []
    }

    # 사용자의 주요 특성별로 추천 영화 분류
    for _, row in df.iterrows():
        user_profile = row.get('user_profile')
        recs = row.get('recommendation_candidates', [])

        if user_profile and 'primary_feature' in user_profile:
            primary_feature = user_profile['primary_feature']

            for rec in recs:
                if isinstance(rec, dict):
                    movie_title = rec.get('title', 'Unknown')
                    feature_movies[primary_feature].append(movie_title)

    # 각 Feature별 인기 영화 출력
    for feature, movies in feature_movies.items():
        if movies:
            movie_counts = pd.Series(movies).value_counts()
            print(f"🎯 {feature} 중심 사용자들이 선호하는 영화:")
            for title, count in movie_counts.head(5).items():
                print(f"  {title}: {count}회")
            print()

# **메인 분석 함수**
def main_movie_frequency_analysis():
    """
    영화별 추천 횟수 메인 분석
    """
    try:
        # 결과 로드 시도
        result_path = '/content/drive/MyDrive/데사경영학술제/data/final_complete_recommendations.pkl'

        try:
            final_df = pd.read_pickle(result_path)
            print("✅ 저장된 결과 파일 로드 성공")
        except:
            # 현재 세션의 결과 사용
            if 'debug_results' in globals():
                final_df = pd.DataFrame(debug_results)
                print("✅ 현재 세션 결과 사용")
            else:
                print("❌ 분석할 데이터가 없습니다.")
                print("추천 시스템을 먼저 실행해주세요.")
                return

        total_users = len(final_df)
        print(f"분석 대상: {total_users}명의 사용자\n")

        # 1. 영화별 추천 횟수 분석
        recommendation_counts, movie_details = analyze_movie_recommendation_frequency(final_df)

        if recommendation_counts is not None:
            print("\n" + "="*60)

            # 2. 추천 다양성 분석
            analyze_recommendation_diversity(recommendation_counts, total_users)

            print("\n" + "="*60)

            # 3. Feature별 인기 영화 분석
            analyze_popular_movies_by_feature(final_df)

            return recommendation_counts, movie_details

    except Exception as e:
        print(f"❌ 분석 중 오류 발생: {e}")
        return None

# **실행**
print("=== 영화별 추천 횟수 분석 시작 ===\n")
result = main_movie_frequency_analysis()


=== 영화별 추천 횟수 분석 시작 ===

✅ 저장된 결과 파일 로드 성공
분석 대상: 1697명의 사용자

=== 영화별 추천 횟수 분석 ===

📊 전체 통계:
  총 추천 횟수: 8485회
  고유 영화 수: 110개
  평균 추천 횟수: 77.14회/영화

🎬 영화별 추천 횟수 순위:
 1. Sense And Sensibility
    추천 횟수: 1290회
    감독: Ang Lee
    평균 별점: 4.800000190734863
    영화 ID: 434

 2. Bridge on the River Kwai, The
    추천 횟수: 913회
    감독: David Lean
    평균 별점: 4.800000190734863
    영화 ID: 1283

 3. Gaslight (1944)
    추천 횟수: 904회
    감독: George Cukor
    평균 별점: 4.699999809265137
    영화 ID: 1973

 4. Downton Abbey
    추천 횟수: 828회
    감독: Brian Percival, Andy Goddard, Jeremy Webb, David Evans
    평균 별점: 4.900000095367432
    영화 ID: 412

 5. The Tuskegee Airmen
    추천 횟수: 657회
    감독: Robert Markowitz
    평균 별점: 4.800000190734863
    영화 ID: 1202

 6. A Christmas Story
    추천 횟수: 575회
    감독: Bob Clark
    평균 별점: 4.800000190734863
    영화 ID: 296

 7. Gaslight (1944)
    추천 횟수: 321회
    감독: George Cukor
    평균 별점: 4.699999809265137
    영화 ID: 392

 8. Detective Anna
    추천 횟수: 313회
    감독: Feliks Gerchik